<a href="https://colab.research.google.com/github/wonjae124/2021_Lab_project/blob/main/1.Custom_dataset_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#출처 : https://github.com/developer0hye/Custom-CNN-based-Image-Classification-in-PyTorch/blob/master/main.py 
import os 
from PIL import Image

import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torchvision import transforms


In [2]:
class CustomImageDataset(Dataset):
  def read_data_set(self):

    all_img_files = []
    all_labels = []

    class_names = os.walk(self.data_set_path).__next__()[1]

    for index, class_name in enumerate(class_names):
      label = index
      img_dir = os.path.join(self.data_set_path, class_name)
      img_files = os.walk(img_dir).__next__()[2]

      for img_file in img_files:
        img_file = os.path.join(img_dir, img_file)
        img = Image.open(img_file)
        if img is not None:
          all_img_files.append(img_file)
          all_labels.append(label)

    return all_img_files, all_labels, len(all_img_files), len(class_names)

  def __init__(self, data_set_path, transforms=None):
    self.data_set_path = data_set_path
    self.image_files_path, self.labels, self.length, self.num_classes = self.read_data_set()
    self.transforms = transforms

  def __getitem__(self, index):
    image = Image.open(self.image_files_path[index])
    image = image.convert("RGB") #convert가 내장함수인가?

    if self.transforms is not None:
       image = self.transforms(image)
    return {'image':image, 'label':self.labels[index]}
  
  def __len__(self):
    return self.length

In [3]:
class CustomConvNet(nn.Module):
  def __init__(self, num_classes):
    super(CustomConvNet, self).__init__()

    self.layer1 = self.conv_module(3, 16)
    self.layer2 = self.conv_module(16, 32)
    self.layer3 = self.conv_module(32, 64)
    self.layer4 = self.conv_module(64, 128)
    self.layer5 = self.conv_module(128, 256)
    self.gap = self.global_avg_pool(256, num_classes)

  def forward(self,x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = self.layer3(out)
    out = self.layer4(out)
    out = self.layer5(out)
    out = self.gap(out)
    out = out.view(-1, num_classes)

    return out

  def conv_module(self, in_num, out_num):
    return nn.Sequential(
        nn.Conv2d(in_num,out_num,kernel_size=3, stride =1, padding =1),
        nn.BatchNorm2d(out_num),
        nn.LeakyReLU(),
        nn.MaxPool2d(kernel_size = 2, stride = 2))
    
  def global_avg_pool(self, in_num, out_num):
    return nn.Sequential(
        nn.Conv2d(in_num,out_num,kernel_size=3, stride=1,padding=1),
        nn.BatchNorm2d(out_num),
        nn.LeakyReLU(),
        nn.AdaptiveAvgPool2d((1,1)))
    

In [5]:
hyper_param_epoch = 20
hyper_param_batch = 8
hyper_param_learning_rate = 0.001

transforms_train = transforms.Compose([transforms.Resize((128,128)),
                                       transforms.RandomRotation(10.),
                                       transforms.ToTensor()])

transforms_test = transforms.Compose([transforms.Resize((128,128)),
                                      transforms.ToTensor()])

train_data_set = CustomImageDataset(data_set_path="/content/drive/MyDrive/Colab Notebooks/train", transforms=transforms_train)
train_loader = DataLoader(train_data_set, batch_size=hyper_param_batch, shuffle=True)

test_data_set = CustomImageDataset(data_set_path="/content/drive/MyDrive/Colab Notebooks/test", transforms=transforms_test)
test_loader = DataLoader(test_data_set, batch_size = hyper_param_batch, shuffle=True)

if not(train_data_set.num_classes == test_data_set.num_classes):
   print("error:Numbers of class in training set and test set are not equal")
   exit()

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')



In [6]:
print(len(train_data_set))
print(len(test_data_set))

print(train_loader)

100
20


In [17]:
num_classes = train_data_set.num_classes
custom_model = CustomConvNet(num_classes=num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(custom_model.parameters(), lr = hyper_param_learning_rate)


In [19]:

for e in range(hyper_param_epoch):
    for i_batch, item in enumerate(train_loader):
      images = item['image'].to(device)
      labels = item['label'].to(device)

      outputs = custom_model(images)
      loss = criterion(outputs, labels)

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      if (i_batch+1)%hyper_param_batch == 0:
        print('Epoch [{}/{}],Loss: {:.4f}'
              .format(e+1, hyper_param_epoch,loss.item()))

Epoch [1/20],Loss: 0.4673
Epoch [2/20],Loss: 0.2165
Epoch [3/20],Loss: 0.2710
Epoch [4/20],Loss: 0.2022
Epoch [5/20],Loss: 0.2068
Epoch [6/20],Loss: 0.1971
Epoch [7/20],Loss: 0.1931
Epoch [8/20],Loss: 0.3453
Epoch [9/20],Loss: 0.2115
Epoch [10/20],Loss: 0.2035
Epoch [11/20],Loss: 0.2012
Epoch [12/20],Loss: 0.1942
Epoch [13/20],Loss: 0.1915
Epoch [14/20],Loss: 0.1871
Epoch [15/20],Loss: 0.1853
Epoch [16/20],Loss: 0.7566
Epoch [17/20],Loss: 0.2015
Epoch [18/20],Loss: 0.1861
Epoch [19/20],Loss: 0.3363
Epoch [20/20],Loss: 0.1593


In [14]:
custom_model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for item in test_loader:
        images = item['image'].to(device)
        labels = item['label'].to(device)
        outputs = custom_model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += len(labels)
        correct += (predicted == labels).sum().item()
        
    print('Test Accuracy of the model on the {} test images: {} %'.format(total, 100 * correct / total))

tensor([False, False, False, False, False, False, False, False])
tensor([False, False, False, False, False, False, False, False])
tensor([False, False, False, False])
Test Accuracy of the model on the 20 test images: 0.0 %
